In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_profit_ratio")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   5759 non-null   int64              
 1   ticker               5759 non-null   object             
 2   stac_yymm            5759 non-null   object             
 3   cptl_ntin_rate       5759 non-null   object             
 4   self_cptl_ntin_inrt  5759 non-null   object             
 5   sale_ntin_rate       5759 non-null   object             
 6   sale_totl_rate       5759 non-null   object             
 7   updated_at           5759 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 360.1+ KB


,id,ticker,stac_yymm,cptl_ntin_rate,self_cptl_ntin_inrt,sale_ntin_rate,sale_totl_rate,updated_at
0,1,005930,202506,5.23,6.64,8.68,34.89,2025-09-16 13:02:19.322178+00:00
1,2,005930,202503,6.38,8.16,10.39,35.55,2025-09-16 13:02:19.322178+00:00
2,3,005930,202412,7.10,9.03,11.45,37.99,2025-09-16 13:02:19.322178+00:00
3,4,005930,202409,7.52,9.52,11.86,38.11,2025-09-16 13:02:19.322178+00:00
4,5,005930,202406,7.05,8.95,11.37,38.21,2025-09-16 13:02:19.322178+00:00
...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,2.88,7.95,2.10,8.04,2025-09-16 13:02:19.322178+00:00
5755,5756,002710,201812,3.34,9.19,2.36,8.54,2025-09-16 13:02:19.322178+00:00
5756,5757,002710,201809,2.81,7.76,1.99,7.57,2025-09-16 13:02:19.322178+00:00
5757,5758,002710,201806,2.04,5.68,1.47,7.38,2025-09-16 13:02:19.322178+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df)

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)


# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
    'missing_count': missing_value_counts,
    'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 5].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 7] - [결측 컬럼 : 0] = [후보 컬럼 : 7]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['updated_at']
[원본 컬럼 : 7] - [결측 컬럼 : 0] - [유니크 컬럼 : 1] = [후보 컬럼 : 6]


In [7]:
df[final_cols]

,ticker,stac_yymm,cptl_ntin_rate,self_cptl_ntin_inrt,sale_ntin_rate,sale_totl_rate
0,005930,202506,5.23,6.64,8.68,34.89
1,005930,202503,6.38,8.16,10.39,35.55
2,005930,202412,7.10,9.03,11.45,37.99
3,005930,202409,7.52,9.52,11.86,38.11
4,005930,202406,7.05,8.95,11.37,38.21
...,...,...,...,...,...,...
5754,002710,201903,2.88,7.95,2.10,8.04
5755,002710,201812,3.34,9.19,2.36,8.54
5756,002710,201809,2.81,7.76,1.99,7.57
5757,002710,201806,2.04,5.68,1.47,7.38


In [8]:
print(final_cols)

['ticker', 'stac_yymm', 'cptl_ntin_rate', 'self_cptl_ntin_inrt', 'sale_ntin_rate', 'sale_totl_rate']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 재무 지표인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `stac_yymm` | 결산년월 | **필수** | 데이터가 어느 시점의 실적인지를 나타내는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 이벤트와 결합할 때 필수적입니다. |
| `cptl_ntin_rate`| 총자본순이익률 | **상** | 기업이 총자본(부채+자본)을 얼마나 효율적으로 사용해 순이익을 냈는지를 보여주는 지표입니다. 기업의 전반적인 수익 창출 능력을 나타냅니다. |
| `self_cptl_ntin_inrt`| 자기자본순이익률 | **상** | ROE(Return on Equity)와 동일한 의미로, 주주의 돈(자기자본)으로 얼마나 많은 이익을 창출했는지를 나타내는 핵심 수익성 지표입니다. |
| `sale_ntin_rate`| 매출액순이익률 | **상** | 매출액 대비 순이익의 비율로, 기업의 최종적인 마진율(수익성)을 보여줍니다. 이 비율의 개선은 주가에 긍정적인 영향을 미칩니다. |
| `sale_totl_rate`| 총자본회전율 | **중** | 기업이 가진 총자본을 얼마나 활발하게 사용하여 매출을 일으켰는지를 나타내는 효율성 지표입니다. 이익의 '양'보다는 '효율'을 측정합니다. |

In [9]:
final_cols = [
  'ticker',
  'stac_yymm',
  'cptl_ntin_rate',
  'self_cptl_ntin_inrt',
  'sale_ntin_rate',
  'sale_totl_rate'
]

In [10]:
df[final_cols]

,ticker,stac_yymm,cptl_ntin_rate,self_cptl_ntin_inrt,sale_ntin_rate,sale_totl_rate
0,005930,202506,5.23,6.64,8.68,34.89
1,005930,202503,6.38,8.16,10.39,35.55
2,005930,202412,7.10,9.03,11.45,37.99
3,005930,202409,7.52,9.52,11.86,38.11
4,005930,202406,7.05,8.95,11.37,38.21
...,...,...,...,...,...,...
5754,002710,201903,2.88,7.95,2.10,8.04
5755,002710,201812,3.34,9.19,2.36,8.54
5756,002710,201809,2.81,7.76,1.99,7.57
5757,002710,201806,2.04,5.68,1.47,7.38


In [11]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ticker               5759 non-null   category      
 1   stac_yymm            5759 non-null   datetime64[ns]
 2   cptl_ntin_rate       5759 non-null   object        
 3   self_cptl_ntin_inrt  5759 non-null   object        
 4   sale_ntin_rate       5759 non-null   object        
 5   sale_totl_rate       5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 246.0+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 티커(종목 코드) | `category` | `category` | 반복적으로 나타나는 종목 코드는 메모리 효율을 위해 `category` 타입으로 관리하는 것이 가장 좋습니다. |
| `stac_yymm` | 결산년월 | `datetime64[ns]` | `datetime64[ns]` | `YYYYMM` 형식의 날짜 데이터는 `pd.to_datetime` 사용 시 `format='%Y%m'`을 지정하여 `datetime`으로 변환해야 합니다. |
| `cptl_ntin_rate`| 총자본순이익률 | `object` | `float64` | 소수점을 포함하는 비율 데이터이므로, 부동소수점 타입인 `float64`로 변환해야 정확한 연산이 가능합니다. |
| `self_cptl_ntin_inrt`| 자기자본순이익률 | `object` | `float64` | 다른 수익성 비율과 마찬가지로 소수점을 포함하는 숫자이므로 `float64` 타입이 적합합니다. |
| `sale_ntin_rate`| 매출액순이익률 | `object` | `float64` | 데이터의 일관성과 정확한 계산을 위해 다른 비율 데이터와 동일하게 `float64` 타입으로 관리해야 합니다. |
| `sale_totl_rate`| 총자본회전율 | `object` | `float64` | 소수점을 포함하는 효율성 지표이므로 `float64` 타입으로 변환해야 합니다. |

In [13]:
cols_to_check = ['cptl_ntin_rate', 'self_cptl_ntin_inrt', 'sale_ntin_rate', 'sale_totl_rate']

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result.round(2))

cptl_ntin_rate         0.03
self_cptl_ntin_inrt    0.03
sale_ntin_rate         0.01
sale_totl_rate         0.08
dtype: float64


In [14]:
dtype_map = {
  'float64': ['cptl_ntin_rate', 'self_cptl_ntin_inrt', 'sale_ntin_rate', 'sale_totl_rate']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ticker               5759 non-null   category      
 1   stac_yymm            5759 non-null   datetime64[ns]
 2   cptl_ntin_rate       5759 non-null   float64       
 3   self_cptl_ntin_inrt  5759 non-null   float64       
 4   sale_ntin_rate       5759 non-null   float64       
 5   sale_totl_rate       5759 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(4)
memory usage: 246.0 KB
